In [2]:
import os.path
import shutil

import numpy as np
import pandas as pd

df = pd.read_csv("data resources/sGSL-Northumberland-Survey-FGP.csv", encoding="windows-1252")
df

,cruise_number__numero_de_croisiere,year__annee,month__mois,day_jour,start_hour__heure_de_depart,start_minute__minute_de_depart,latitude,longitude,gear__equipement,species__espece,french_name__nom_francais,english_name__nom_anglais,latin_name__nom_latin,weight_caught__poids_pris,number_caught__quantite_attrape
0,24,2000,7,25,10.0,39.0,46.774167,-64.750917,No. 286 otter trawl with rockhopper,43,Plie rouge,Winter flounder,Pseudopleuronectes americanus,0.500,1.0
1,24,2000,7,25,10.0,39.0,46.774167,-64.750917,No. 286 otter trawl with rockhopper,122,Tanche-tautogue,Cunner,Tautogolabrus adspersus,0.800,2.0
2,24,2000,7,25,10.0,39.0,46.774167,-64.750917,No. 286 otter trawl with rockhopper,2550,Homard américain,American lobster,Homarus americanus,3.800,16.0
3,24,2000,7,25,10.0,39.0,46.774167,-64.750917,No. 286 otter trawl with rockhopper,4330,Moule non-spécifié,Mussel unidentified,Mytilidae (f.),0.010,1.0
4,24,2000,7,25,10.0,39.0,46.774167,-64.750917,No. 286 otter trawl with rockhopper,6200,Ophiuroidea non-spécifié,Brittle star unidentified,Ophiuroidea (c.),0.010,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29950,303,2023,8,1,14.0,21.0,45.933583,-62.195333,Northumberland trawl,623,Lompénie tachetée,Daubed shanny,Leptoclinus maculatus,0.003,1.0
29951,303,2023,8,1,14.0,21.0,45.933583,-62.195333,Northumberland trawl,2212,Crevette ésope,Aesop shrimp,Pandalus montagui,0.004,NaN
29952,303,2023,8,1,14.0,21.0,45.933583,-62.195333,Northumberland trawl,2411,Crevette verte,Arctic argid,Argis dentata,0.185,NaN
29953,303,2023,8,1,14.0,21.0,45.933583,-62.195333,Northumberland trawl,2550,Homard américain,American lobster,Homarus americanus,0.122,1.0


In [3]:
# define the function to combine datetime from a row
def combine_date(row):
    year = str(row["year__annee"]).zfill(4)
    month = str(row["month__mois"]).zfill(2)
    day = str(row["day_jour"]).zfill(2)
    hour = row["start_hour__heure_de_depart"]
    minute = row["start_minute__minute_de_depart"]
    if (not isinstance(hour, str) and np.isnan(hour)) or (not isinstance(minute, str) and np.isnan(minute)):
        return f"{year}-{month}-{day}"
    else:
        return f"{year}-{month}-{day} {int(hour)}:{int(minute)}"

df.apply(combine_date, axis=1)

0        2000-07-25 10:39
1        2000-07-25 10:39
2        2000-07-25 10:39
3        2000-07-25 10:39
4        2000-07-25 10:39
               ...       
29950    2023-08-01 14:21
29951    2023-08-01 14:21
29952    2023-08-01 14:21
29953    2023-08-01 14:21
29954    2023-08-01 14:21
Length: 29955, dtype: object

In [4]:
# add a new column that is a datetime
df["datetime"] = pd.to_datetime(df.apply(combine_date, axis=1))
df_trunc = df.sort_values("datetime").loc[:, ["year__annee", "latitude", "longitude", "datetime"]]
df_trunc = df_trunc.drop_duplicates()
df_trunc

,year__annee,latitude,longitude,datetime
0,2000,46.774167,-64.750917,2000-07-25 10:39:00
6,2000,46.803500,-64.804250,2000-07-25 11:27:00
11,2000,46.838167,-64.837333,2000-07-25 12:40:00
19,2000,46.869500,-64.840333,2000-07-25 20:29:00
20,2000,46.895083,-64.795417,2000-07-26 08:06:00
...,...,...,...,...
29914,2023,46.245250,-62.138667,2023-07-31 15:37:00
29925,2023,46.193500,-62.242167,2023-08-01 08:27:00
29935,2023,46.072000,-62.427667,2023-08-01 10:21:00
29944,2023,46.035250,-62.191083,2023-08-01 12:57:00


In [6]:
from stock_assessment_surveys.utils.xy2line import xy2line
xy2line(df_trunc)